<a href="https://colab.research.google.com/github/DkCodeProjct/AI_Projects..-/blob/main/Ai_projects5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np
%matplotlib inline
import random

In [ ]:



class Value:
    def __init__(self, data, _childern=(), _op='', label=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None  # default is a no-op
        self._prev = set(_childern)    # previous nodes in the graph
        self._op = _op                 # operation that produced this node
        self.label = label             # node label for visualization

    def __repr__(self):
        return f'Value(data={self.data}, label={self.label})'

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        output = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * output.grad
            other.grad += 1.0 * output.grad
        output._backward = _backward
        return output

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        output = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * output.grad
            other.grad += self.data * output.grad
        output._backward = _backward
        return output


    # Power Rule:
    #     d/dx*x**n = n*x**n-1
    def __pow__(self, other):
        assert isinstance(other, (int, float)),  "only suport int/flot for now"
        out = Value(self.data ** other, (self, ), f'**{other}')

        def _backward():
            self.grad = (other * self.data**(other-1)) * out.grad # power rule
        out._backward = _backward
        return out


    def __rmul__(self, other):
        return self * other


    def __truediv__(self, other):
        return self * other**-1


    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)


    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        output = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * output.grad
        output._backward = _backward
        return output


    def expo(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.grad * out.data
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()

        # Build the topological order
        def buildTopo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    buildTopo(child)
                topo.append(v)

        buildTopo(self)


        self.grad = 1
        for v in reversed(topo):
            v._backward()





In [ ]:
from graphviz import Digraph


In [ ]:

def trace(rt):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(rt)
    return nodes, edges
def drawDot(rt):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
    nodes, edges = trace(rt)

    for n in nodes:
        vid = str(id(n))
        # Check if the label exists and format it properly
        label_text = f'{n.label} | data {n.data:.4f} | grad {n.grad:.4f}' if n.label else f'data {n.data:.4f} | grad {n.grad:.4f}'

        dot.node(name=vid, label=f'{{{label_text}}}', shape='record')

        if n._op:
            dot.node(name=vid + n._op, label=f'{n._op}')
            dot.edge(vid + n._op, vid)

    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot


In [ ]:


class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        dot = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = dot.tanh()
        return out


    def parameters(self):
        return self.w + [self.b]


class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out
    def parameters(self):

        return [p for neurn in self.neurons for p in neurn.parameters()]
        #para = []
        #for nurn in self.neurons:
        #    ps = nurn.parameters()
        #    para.extend(ps)
        #return para

class MLP:
    def __init__(self, nin, nout):
        sz = [nin] + nout
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nout))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):

        return [p for layr in self.layers for p in layr.parameters()]



In [ ]:

x = [2.0, 3.0, -1.0]
n = MLP(3, [4,4,1])
n(x)

Value(data=0.7619334742780517, label=)

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0] # Desired output // test data
ypred = [n(x) for x in xs]
ypred

[Value(data=0.7619334742780517, label=),
 Value(data=-0.47078859910865517, label=),
 Value(data=0.6051824696364113, label=),
 Value(data=0.7325122130510542, label=)]

In [ ]:
n.layers[0].neurons[0].w[0].grad


0

In [ ]:
loss = sum(((yPrd - Value(yTrue))**2 for yTrue, yPrd in zip(ys, ypred)), start=Value(0))

loss


Value(data=2.9849008544975915, label=)

In [ ]:
for k in range(20):
    ypred = [n(x) for x in xs]
    loss = sum(((yPrd - Value(yTrue))**2 for yTrue, yPrd in zip(ys, ypred)), start=Value(0))

    # set grad to 0 cos it has to reset in evry iteration
    for p in n.parameters():
        p.grad = 0.0

    loss.backward()

    for p in n.parameters():
        p.data += -0.1 * p.grad

    print(k, loss.data)

0 0.06684021555487227
1 0.05715907937027989
2 0.04964023414284956
3 0.04366705937179049
4 0.03883068130225818
5 0.034850732172899274
6 0.03152942071683999
7 0.028723760432398736
8 0.026328164873588526
9 0.02426319750146002
10 0.022468100567626527
11 0.020895713909282854
12 0.019508945112155215
13 0.018278270402588642
14 0.01717993478671285
15 0.01619463556822181
16 0.015306545768556407
17 0.01450258030472023
18 0.013771838028290807
19 0.013105172839705831
